In [5]:
#impoering the libraries and moduels
import torch
import torch.nn as nn
from torch.utils.data import Dataset ,DataLoader
from sklearn.model_selection import train_test_split
from torchsummary import summary
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.optim import Adam

device = 'cuda' if torch.cuda.is_available() else 'cpu'



In [6]:
#importing and cleaning the dataset
test_data_df = pd.read_csv(r"D:\Datasets\Kaggle\Driving _behaviour_dataset\test_motion_data.csv")
train_data_df = pd.read_csv(r"D:\Datasets\Kaggle\Driving _behaviour_dataset\train_motion_data.csv")

In [15]:
original_test_data_df = test_data_df.copy()
original_train_data_df = train_data_df.copy()
test_data_df.dropna(inplace=True)
train_data_df.dropna(inplace=True)



In [16]:
#normalizing 
for column in train_data_df:
    if train_data_df[column].dtype in  ['int','float']:
        train_data_df[column] = train_data_df[column]/train_data_df[column].abs().max()

for column in test_data_df:
    if test_data_df[column].dtype in ['int','float']:
        test_data_df[column] = test_data_df[column]/test_data_df[column].abs().max()


In [21]:
#omitting out the timestamp column cuz its irrelevant
test_data_df.drop(['Timestamp'], axis=1, inplace=True)
train_data_df.drop(['Timestamp'], axis=1, inplace=True)
print(train_data_df.head())

       AccX      AccY      AccZ     GyroX     GyroY     GyroZ   Class
0  0.000000  0.000000  0.000000  0.069951 -0.104000  0.082443  NORMAL
1 -0.325915 -0.230327 -0.028581 -0.033627  0.030545  0.109615  NORMAL
2 -0.119277 -0.026046  0.030865 -0.022838 -0.017455  0.071080  NORMAL
3  0.148124 -0.048610  0.093468  0.082179 -0.017818  0.044402  NORMAL
4  0.020407  0.165447 -0.009341  0.036145 -0.002182  0.044402  NORMAL


## Current Workflow ##


1. Data cleaning - done
2. Dataset setup: most conceptual i believe
3. DataLoader magik
4. Design the model, ooh gawd
5. Training loop ooh lawd have mercy
6. Test the model, ** insert confused cat meme here **

In [22]:
###Dataset magik

#converting to numpy arrays
X = np.array(train_data_df.iloc[:,:-1])
Y = np.array(train_data_df.iloc[:,-1])

#creating a sweet validation set from train set
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, train_size=0.8)

class dataset(Dataset):
    def __init__(self, X, Y):
        self.X = torch.tensor(X, dtype=float, requires_grad=True).to(device)
        self.Y = torch.tensor(Y, dtype=float, requires_grad=True).to(device)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index], self.Y[index]
    